In [1]:

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
# Importing some important libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline



In [2]:
import pandas as pd
import boto3

# created a bucket named "damg-project" with "card_transdata.csv" file in it

bucket = "damg-project"
file_name = "card_transdata.csv"


s3 = boto3.client('s3') 
# 's3' is a key word. create connection to S3 using default config and all buckets within S3

obj = s3.get_object(Bucket= bucket, Key= file_name) 
# get object and file (key) from bucket

initial_df = pd.read_csv(obj['Body']) # 'Body' is a key word





In [4]:
print(initial_df.head)
print(" Shape of the dataset: ", initial_df.shape)

<bound method NDFrame.head of         distance_from_home  distance_from_last_transaction  \
0                57.877857                        0.311140   
1                10.829943                        0.175592   
2                 5.091079                        0.805153   
3                 2.247564                        5.600044   
4                44.190936                        0.566486   
...                    ...                             ...   
999995            2.207101                        0.112651   
999996           19.872726                        2.683904   
999997            2.914857                        1.472687   
999998            4.258729                        0.242023   
999999           58.108125                        0.318110   

        ratio_to_median_purchase_price  repeat_retailer  used_chip  \
0                             1.945940              1.0        1.0   
1                             1.294219              1.0        0.0   
2              

In [5]:

print(initial_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   distance_from_home              1000000 non-null  float64
 1   distance_from_last_transaction  1000000 non-null  float64
 2   ratio_to_median_purchase_price  1000000 non-null  float64
 3   repeat_retailer                 1000000 non-null  float64
 4   used_chip                       1000000 non-null  float64
 5   used_pin_number                 1000000 non-null  float64
 6   online_order                    1000000 non-null  float64
 7   fraud                           1000000 non-null  float64
dtypes: float64(8)
memory usage: 61.0 MB
None


In [6]:
initial_df.columns

Index(['distance_from_home', 'distance_from_last_transaction',
       'ratio_to_median_purchase_price', 'repeat_retailer', 'used_chip',
       'used_pin_number', 'online_order', 'fraud'],
      dtype='object')

In [7]:
initial_df.isnull().sum()


distance_from_home                0
distance_from_last_transaction    0
ratio_to_median_purchase_price    0
repeat_retailer                   0
used_chip                         0
used_pin_number                   0
online_order                      0
fraud                             0
dtype: int64

In [8]:
percent = (len(initial_df.loc[initial_df.fraud==1])) / (len(initial_df.loc[initial_df
                                                                           .fraud == 0])) * 100
print(f"Percentage of Fraudulent Transanctions in the Dataset: {percent}%")


Percentage of Fraudulent Transanctions in the Dataset: 9.577392868922427%


In [3]:
X = initial_df.drop('fraud', axis=1)
y = initial_df.fraud

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)


     


In [5]:
lr = LogisticRegression()
model = lr.fit(X_train, y_train)
 
# Predict on training set
lr_pred = model.predict(X_test)


In [12]:
s3_resource = boto3.resource('s3')
import pickle

bucket='damg-model'
key= 'pickle_model.pkl'

pickle_byte_obj = pickle.dumps(model)

s3_resource.Object(bucket,key).put(Body=pickle_byte_obj)

{'ResponseMetadata': {'RequestId': '85TZ44282HSZ7CN7',
  'HostId': 'jv7BGOoM7oZFfw9ufkreVPqmVVxpDsWsiayngNkwsfnhaANDALP4jZWh7xS22S/8oJ1jTqw7VZ2CMaO7/dDyxA==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'jv7BGOoM7oZFfw9ufkreVPqmVVxpDsWsiayngNkwsfnhaANDALP4jZWh7xS22S/8oJ1jTqw7VZ2CMaO7/dDyxA==',
   'x-amz-request-id': '85TZ44282HSZ7CN7',
   'date': 'Fri, 09 Dec 2022 21:57:30 GMT',
   'etag': '"367a5a6c456e6f766d3211af02753ad5"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"367a5a6c456e6f766d3211af02753ad5"'}

In [9]:
# import joblib
# joblib.dump(model, "model.pkl")


['model.pkl']

In [17]:
lr_pred = model.predict([[2,3,42,0.3,3,1,3]])
print(lr_pred)

[1.]
